In [2]:
import tensorflow as tf
from icecream import ic
from threading import Thread
import numpy as np
from Bio import pairwise2
import time

Here I am tring to make two tensors shift to make the alignment. I need max(len) - 1 iteration which can be converted in n/2, because I can shift both extremities. However the issue now is that the two tensor are not the same iteration.

I want as results a tensor long as the number of total possiple comparison between the two sequences. The value will be representative as the sum of the differences between the two tensors, which are number converted from the bases. Choosing the minimum sun is as choosing the best allignment. Here rise the proble that different bases relate different numbe and therefore different penalties when mismach are found. In contranst is possible, but risky saing that since long reads will be preferably used, this consequence will be dumpened from the length of the reads.

Ireally need to keep all of them in a tensor or maybe I only need the min value and the index of it???

In [79]:
seq1 = "ATCATATGCA"
seq2 = "ATC"

tf1 = tf.constant([ord(c) for c in seq1])
tf2 = tf.constant([ord(c) for c in seq2])

min_shift_tensor = min(tf1.shape[0], tf2.shape[0])
max_shape_tensor = max(tf1.shape[0], tf2.shape[0])
num_iterations = (max_shape_tensor + min_shift_tensor - 1) / 2
int_num_iteration = (max_shape_tensor + min_shift_tensor - 1) // 2

ic(num_iterations)
ic(int_num_iteration)

alone = False # This is for the 4.5 iteration

if int_num_iteration < num_iterations:
    alone = True

print(f"Max lenght of the two tensors : {max_shape_tensor}, the min: {min_shift_tensor}")
print(f"Tensor one in his completness: {tf1}")
print(f"Tensor two in his completness: {tf2}\n")
ic(alone)

for i in range(int_num_iteration):
    if i < min_shift_tensor:
        print(f"Iteration : {i}")
        print(f"Tensor one : {tf1[:(i+1)]}")
        print(f"Tensor two : {tf2[-(i+1):]}\nThe opposite frame work")
        print(f"Tensor one : {tf1[-(i+1):]}")
        print(f"Tensor two : {tf2[:(i+1)]}\n")
    if i >= min_shift_tensor:
        print(f"Iteration : {i}")
        print(f"Tensor one : {tf1[i-min_shift_tensor+1:(i+1)]}")
        print(f"Tensor two : {tf2[-(i+1):]}\nThe opposite frame work")
        print(f"Tensor one : {tf1[-(i+1):-(i-min_shift_tensor+1)]}")
        print(f"Tensor two : {tf2[:(i+1)]}\n")
    if (i == int_num_iteration-1) and alone:
        i += 1
        print("Extra single iteration")
        print(f"Iteration : {i}")
        print(f"Tensor one : {tf1[i-min_shift_tensor+1:(i+1)]}")
        print(f"Tensor two : {tf2[-(i+1):]}")

ic| num_iterations: 6.0
ic| int_num_iteration: 6
ic| alone: False


Max lenght of the two tensors : 10, the min: 3
Tensor one in his completness: [65 84 67 65 84 65 84 71 67 65]
Tensor two in his completness: [65 84 67]

Iteration : 0
Tensor one : [65]
Tensor two : [67]
The opposite frame work
Tensor one : [65]
Tensor two : [65]

Iteration : 1
Tensor one : [65 84]
Tensor two : [84 67]
The opposite frame work
Tensor one : [67 65]
Tensor two : [65 84]

Iteration : 2
Tensor one : [65 84 67]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [71 67 65]
Tensor two : [65 84 67]

Iteration : 3
Tensor one : [84 67 65]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [84 71 67]
Tensor two : [65 84 67]

Iteration : 4
Tensor one : [67 65 84]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [65 84 71]
Tensor two : [65 84 67]

Iteration : 5
Tensor one : [65 84 65]
Tensor two : [65 84 67]
The opposite frame work
Tensor one : [84 65 84]
Tensor two : [65 84 67]



In [5]:
from numba import jit
import numpy as np

@jit(nopython = True)
def np_align_func(sequence_one:str, sequence_two:str, match:int = 3, mismatch:int = -2):
    """Do something
    """

    # Initialization of output, and since the the func return only one vaue for each the check will be if the saved value is greater or not
    score = 0
    diff = 0
    switch = False

    # String are transponed in number using the ord() func, tha aim is to use mathematichs instead of comparison between strigns.
    seq_one = np.array([ord(c) for c in sequence_one])
    seq_two = np.array([ord(c) for c in sequence_two])

    # This is neceesary since knowing which one in the longest is also needed. seq_one is now scurely the longest
    if seq_one.shape[0] >= seq_two.shape[0]:
        max_lenght_seq = seq_one.shape[0]
        min_length_seq = seq_two.shape[0]

    else:
        switch = True
        max_lenght_seq = seq_two.shape[0]
        min_length_seq = seq_two.shape[0]
        seq_one, seq_two = seq_two, seq_one
    
    # Number of iterations
    num_iteration_int = (max_lenght_seq + min_length_seq - 1) // 2
    num_iteration = (max_lenght_seq + min_length_seq - 1) / 2
    alone = False

    if num_iteration > num_iteration_int:
        alone = True
    # ic(num_iteration, num_iteration_int, alone)

    for i in range(num_iteration_int):
        if i < min_length_seq:

            align_forw = seq_one[:(i+1)] - seq_two[-(i+1):]
            align_back = seq_two[:(i+1)] - seq_one[-(i+1):]

            cnt = 0
            for j in align_forw, align_back:
                part_score = np.count_nonzero(j == 0)*match + np.count_nonzero(j)*mismatch

                if part_score > score:
                    score = part_score
                    if cnt > 0:
                        # Mean that the longest read (seq_one) is forward respect the second
                        diff = max_lenght_seq -i -1
                    else:
                        # This is the oppoite case, where the shortest is also the forward one
                        diff = -(min_length_seq -i -1)
                cnt += 1
        
        if i >= min_length_seq:
            align_forw = seq_one[i-min_length_seq+1:(i+1)] - seq_two[-(i+1):]
            align_back = seq_one[-(i+1):-(i-min_length_seq+1)] - seq_two[:(i+1)]

            cnt = 0
            for j in align_forw, align_back:
                part_score = np.count_nonzero(j == 0)*match + np.count_nonzero(j)*mismatch

                
                if part_score > score:
                    score = part_score
                    if cnt > 0:
                        diff = max_lenght_seq -i -1
                    else:
                        diff = -(min_length_seq -i -1)
                cnt += 1 

        if i == (num_iteration_int-1) and alone:
            i += 1

            align_forw = seq_one[i-min_length_seq+1:(i+1)] - seq_two[-(i+1):]
            part_score = np.count_nonzero(align_forw == 0)*match + np.count_nonzero(align_forw)*mismatch

            if part_score > score:
                score = part_score
                # TODO think of the frame
                diff = max_lenght_seq -i -1


    return score, diff, switch

s = """TTCCTGGNCTGTAGGTAGCGACTACAGTTAGGGGGCACCTAGCATTCAGGCCCTCATCCTCCTCCTTCCC
AGCAGGGTGTCACGCTTCTCCGAAGACTGGATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCG
AGCTCCCTCTGAGCCAGGAGACATTTTCAGGCTTATGGAAACTACTTCCTCCAGAAGATATCCTGCCATC
ACCTCACTGCATGGACGATCTGTTGCTGCCCCAGGATGTTGAGGAGTTTTTTGAAGGCCCAAGTGAAGCC
CTCCGAGTGTCAGGAGCTCCTGCAGCACAGGACCCTGTCACCGAGACCCCTGGGCCAGTGGCCCCTGCCC
CAGCCACTCCATGGCCCCTGTCATCTTTTGTCCCTTCTCAAAAAACTTACCAGGGCAACTATGGCTTCCA
CCTGGGCTTCCTGCAGTCTGGGACAGCCAAGTCTGTTATGTGCACGTACTCTCCTCCCCTCAATAAGCTA
TTCTGCCAGCTGGCGAAGACGTGCCCTGTGCAGTTGTGGGTCAGCGCCACACCTCCAGCTGGGAGCCGTG
TCCGCGCCATGGCCATCTACAAGAAGTCACAGCACATGACGGAGGTCGTGAGACGCTGCCCCCACCATGA
GCGCTGCTCCGATGGTGATGGCCTGGCTCCTCCCCAGCATCGTATCCGGGTGGAAGGAAATTTGTATCCC
GAGTATCTGGAAGACAGGCAGACTTTTCGCCACAGCGTGGTGGTACCTTATGAGCCACCCGAGGCCGGCT
CTGAGTATACCACCATCCACTACAAGTACATGTGTAATAGCTCCTGCATGGGGGGCATGAACCGCCGACC
TATCCTTACCATCATCACACTGGAAGACTCCAGTGGGAACCTTCTGGGACGGGACAGCTTTGAGGTTCGT
GTTTGTGCCTGCCCTGGGAGAGACCGCCGTACAGAAGAAGAAAATTTCCGCAAAAAGGAAGTCCTTTGCC
CTGAACTGCCCCCAGGGAGCGCAAAGAGAGCGCTGCCCACCTGCACAAGCGCCTCTCCCCCGCAAAAGAA
AAAACCACTTGATGGAGAGTATTTCACCCTCAAGATCCGCGGGCGTAAACGCTTCGAGATGTTCCGGGAG
CTGAATGAGGCCTTAGAGTTAAAGGATGCCCATGCTACAGAGGAGTCTGGAGACAGCAGGGCTCACTCCA
GCTACCTGAAGACCAAGAAGGGCCAGTCTACTTCCCGCCATAAAAAAACAATGGTCAAGAAAGTGGGGCC
TGACTCAGACTGACTGCCTCTGCATCCCGTCCCCATCACCAGCCTCCCCCTCTCCTTGCTGTCTTATGAC
TTCAGGGCTGAGACACAATCCCCAGGATCCACTCANACTGNCTGCCTNTGNATCCNGTCCCCATNACCAN
CCTCCCCNTNTCCTTGCTGNNTTATGACT"""

w = """TTCCTGGNCTGTAGGTAGCGACTACAGTTAGGGGGCACCTAGCATTCAGGCCCTCATCCTCCTCCTTCCC
AGCAGGGTGTCACGCTTCTCCGAAGACTGGATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCG
AGCTCCCTCTGAGCCAGGAGACATTTTCAGGCTTATGGAAACTACTTCCTCCAGAAGATATCCTGCCATC
ACCTCACTGCATGGACGATCTGTTGCTGCCCCAGGATGTTGAGGAGTTTTTTGAAGGCCCAAGTGAAGCC
CTCCGAGTGTCAGGAGCTCCTGCAGCACAGGACCCTGTCACCGAGACCCCTGGGCCAGTGGCCCCTGCCC
CAGCCACTCCATGGCCCCTGTCATCTTTTGTCCCTTCTCAAAAAACTTACCAGGGCAACTATGGCTTCCA
CCTGGGCTTCCTGCAGTCTGGGACAGCCAAGTCTGTTATGTGCACGTACTCTCCTCCCCTCAATAAGCTA
TTCTGCCAGCTGGCGAAGACGTGCCCTGTGCAGTTGTGGGTCAGCGCCACACCTCCAGCTGGGAGCCGTG
TCCGCGCCATGGCCATCTACAAGAAGTCACAGCACATGACGGAGGTCGTGAGACGCTGCCCCCACCATGA
GCGCTGCTCCGATGGTGATGGCCTGGCTCCTCCCCAGCATCGTATCCGGGTGGAAGGAAATTTGTATCCC
GAGTATCTGGAAGACAGGCAGACTTTTCGCCACAGCGTGGTGGTACCTTATGAGCCACCCGAGGCCGGCT
CTGAGTATACCACCATCCACTACAAGTACATGTGTAATAGCTCCTGCATGGGGGGCATGAACCGCCGACC
TATCCTTACCATCATCACACTGGAAGACTCCAGTGGGAACCTTCTGGGACGGGACAGCTTTGAGGTTCGT
GTTTGTGCCTGCCCTGGGAGAGACCGCCGTACAGAAGAAGAAAATTTCCGCAAAAAGGAAGTCCTTTGCC
CTGAACTGCCCCCAGGGAGCGCAAAGAGAGCGCTGCCCACCTGCACAAGCGCCTCTCCCCCGCAAAAGAA
AAAACCACTTGATGGAGAGTATTTCACCCTCAAGATCCGCGGGCGTAAACGCTTCGAGATGTTCCGGGAG
CTGAATGAGGCCTTAGAGTTAAAGGATGCCCATGCTACAGAGGAGTCTGGAGACAGCAGGGCTCACTCCA
GCTACCTGAAGACCAAGAAGGGCCAGTCTACTTCCCGCCATAAAAAAACAATGGTCAAGAAAGTGGGGCC
TGACTCAGACTGACTGCCTCTGCATCCCGTCCCCATCACCAGCCTCCCCCTCTCCTTGCTGTCTTATGAC
TTCAGGGCTGAGACACAATCCCCAGGATCCACTCANACTGNCTGCCTNTGNATCCNGTCCCCATNACCAN
CCTCCCCNTNTCCTTGCTGNNTTATGACT"""

np_align_func(s,w)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<intrinsic length_of_iterator>) found for signature:
 
 >>> length_of_iterator(iter_unicode)
 
There are 2 candidate implementations:
[1m      - Of which 2 did not match due to:
      Intrinsic in function 'length_of_iterator': File: numba\cpython\rangeobj.py: Line 181.
        With argument(s): '(iter_unicode)':[0m
[1m       Rejected as the implementation raised a specific error:
         TypingError: [1mUnsupported iterator found in array comprehension, try preallocating the array and filling manually.[0m[0m
  raised from c:\Users\filoa\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\cpython\rangeobj.py:235
[0m
[0m[1mDuring: resolving callee type: Function(<intrinsic length_of_iterator>)[0m
[0m[1mDuring: typing of call at C:\Users\filoa\AppData\Local\Temp\ipykernel_600\260635289.py (4)
[0m
[1m
File "..\..\..\..\AppData\Local\Temp\ipy

In [15]:
import matplotlib.pyplot as plt
import random

def custom_reads(seq: str, length_reads:int = 160, coverage:int = 5, verbose = False) -> list:
    """The function splits the sequence in input into reads.
    The splitting is done using random numbers, and the number of reads is given by: (len(seq)/length_read)*coverage.
    """

    number_of_reads = int(len(seq)/length_reads) * coverage
    starting_pos = random.sample(range(0, len(seq) - length_reads + 1), number_of_reads)
    reads = []

    for num in starting_pos:
        reads.append(seq[num:num + length_reads])

    if verbose:
        y = [0 for _ in range(0, len(seq) + 1)]
        for i in starting_pos:
            for j in range(i, i + length_reads + 1):
                y[j] += 1

        plt.plot(y)
        plt.xlabel("Position")
        plt.ylabel("Coverage")
        plt.title("Coverage Plot")
        plt.savefig("C:\\Users\\filoa\\OneDrive\\Desktop\\Programming_trials\\Assembler")

        print(f"There are {y.count(0)} bases that have 0 coverage.")

    return reads

tot = s+w

reads = custom_reads(tot, length_reads=40, coverage=12)

In [31]:
f = np.array([1,2,3])
ic(f.shape)


reads = ["ATCCATGCT", "AGCTAGGAGATCGA", "CTAGGATGCATAGG", "TGCATAGGACATAGCTAGGAGATCGAGATCACCAGA", "ATGCATAGGACAT", "TATATGCGCTAG", "AGATATATATGCGC"]
ncpus = 2
split_1 = reads[:len(reads)//2]
split_2 = reads[len(reads)//2:]

ic(split_1)
ic(split_2)

tf_split_1 = tf.ragged.constant([[ord(c) for c in read] for read in split_1])
tf_split_2 = tf.ragged.constant([[ord(c) for c in read] for read in split_2])

# t = Thread(target=,args=)

ic| f.shape: (3,)


ic| split_1: ['ATCCATGCT', 'AGCTAGGAGATCGA', 'CTAGGATGCATAGG']
ic| split_2: ['TGCATAGGACATAGCTAGGAGATCGAGATCACCAGA',
              'ATGCATAGGACAT',
              'TATATGCGCTAG',
              'AGATATATATGCGC']
